Documentation: https://pypi.org/project/twint/

In [5]:
import pandas as pd
import numpy as np
import csv

In [7]:
# Read in 100 users per side
influential_csv = pd.read_csv('influential_tweeters.csv')

In [8]:
influential_csv.head(10)

,affirming,denying
0,algore,skepticscience
1,oxfamgb,ScienceNotDogma
2,globalactplan,Carbongate
3,GreenpeaceUK,Neets2245
4,WWF_Climate,Co2Green
5,OneClimate,thegwpfcom
6,Ed_Miliband,CO2Coalition
7,CarolineLucas,Action4Life_
8,DECCgovuk,fredpalmercoal
9,the_ecologist,NoTricksZone


In [10]:
influential_csv['denying'].iloc[-1]

'DarrinW74512672'

Scrape these users' tweets using Twint:

In [1]:
import twint

In [333]:
c = twint.Config()
c.Show_hashtags = True
c.Count = True
c.Stats = True
c.Store_csv = True
c.Retweets = False
c.Custom["tweet"] = ["id", "date", "time", "tweet", "mentions", "replies_count", "retweets_count", 
                     "likes_count", "hashtags", "username"]
c.Format = "ID {id} -||- Date {date} -||- Time {time} -||- Tweet {tweet} -||- Mentions {mentions} -||- \
Repliescount {replies_count} \
-||- Retweetscount {retweets_count} -||- Likescount {likes_count} -||- Hashtags {hashtags} -||- Username {username}"
sep = " -||- "

In [334]:
curr_affirm = influential_csv['denying'].iloc[0]
print('First affirming user:',curr_affirm)
c.Username = curr_affirm

First affirming user: skepticscience


In [335]:
SEARCH_TERMS = ["global warming","climate change","sea","ice","melt","co2","methane","greenhouse","carbon",
               "emission","anthropogenic","temperature","warming","climate","coal","IPCC","environment",
               "environmental","cap and trade",'energy','planet','renewable','fossil fuel','environmentalist']
# TO DO: expand search_terms w/ some of Zach's terms

In [336]:
c.Username

'skepticscience'

In [374]:
for ix in range(23,len(SEARCH_TERMS)):
    search_term = SEARCH_TERMS[ix]
    print('Getting tweets with keyword: '+search_term)#search_term in SEARCH_TERMS:
    c.Search = search_term
    #c.Pandas = True
    #c.Store_object = True
    c.Output = '{}_search-term={}.csv'.format(c.Username,c.Search)
    twint.run.Search(c)

Getting tweets with keyword: environmentalist


RuntimeError: This event loop is already running

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)


ID 1110212766290378753 -||- Date 2019-03-25 -||- Time 09:12:04 -||- Tweet Senator Mitch McConnell is fighting to bring the Green New Deal up for a vote, but it's not because he's become an environmentalist overnight. #YEARSproject  https://www.facebook.com/SkepticalScience/posts/10156403629468335 … -||- Mentions  -||- Repliescount {replies_count} -||- Retweetscount {retweets_count} -||- Likescount {likes_count} -||- Hashtags #YEARSproject -||- Username skepticscience
ID 1092105203137040385 -||- Date 2019-02-03 -||- Time 08:59:05 -||- Tweet "I have never been an environmentalist. I don’t even think of myself as a nature person. I’ve lived my whole life in...  https://www.theguardian.com/environment/2019/feb/02/the-devastation-of-human-life-is-in-view-what-a-burning-world-tells-us-about-climate-change-global-warming … -||- Mentions  -||- Repliescount {replies_count} -||- Retweetscount {retweets_count} -||- Likescount {likes_count} -||- Hashtags  -||- Username skepticscience
ID 9919354047

Organize scraped Tweets from all searchterms in df; remove duplicates.

In [375]:
user_tweets = []
for search_term in SEARCH_TERMS:
    try:
        tweets_per_search_term = pd.read_csv('{}_search-term={}.csv'.format(c.Username,search_term))
        tweets_per_search_term['search_term'] = search_term
        user_tweets.append(tweets_per_search_term)
    except FileNotFoundError:
        pass

In [376]:
len(user_tweets) # each one is a df

24

In [377]:
user_tweets[-1]

,id,date,time,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,username,search_term
0,1110212766290378753,2019-03-25,09:12:04,Senator Mitch McConnell is fighting to bring t...,[],0,1,2,['#yearsproject'],skepticscience,environmentalist
1,1092105203137040385,2019-02-03,08:59:05,"""I have never been an environmentalist. I don’...",[],3,9,9,[],skepticscience,environmentalist
2,991935404780785664,2018-05-03,00:00:04,The phrase “Mormon environmentalist” is not on...,[],0,1,1,[],skepticscience,environmentalist
3,944387122563354625,2017-12-22,18:00:10,Iceland's new prime minister is a feminist and...,[],1,24,47,[],skepticscience,environmentalist
4,900658957936668672,2017-08-24,03:00:03,"In an upcoming episode of ABC’s Catalyst, Aust...",[],1,6,6,[],skepticscience,environmentalist
5,898393806659358720,2017-08-17,20:59:09,Climate change is the biggest story on any edi...,[],3,13,13,[],skepticscience,environmentalist
6,106601860076544000,2011-08-24,22:40:38,@EileenOttawa thanx Eileen. Don't meet many fe...,['eileenottawa'],0,0,0,[],skepticscience,environmentalist


In [378]:
full_tweets = pd.concat(user_tweets)
full_tweets.shape

(6040, 11)

In [379]:
from collections import Counter
counted_ids = Counter(full_tweets['id'])
print(len([(id_,counted_ids[id_]) for id_ in counted_ids if counted_ids[id_] > 1]))

1395


In [380]:
np.sum([counted_ids[id_] for id_ in counted_ids if counted_ids[id_] > 1])

3202

In [381]:
full_tweets.drop_duplicates(subset ="id", keep = False, inplace = True)

In [382]:
full_tweets.shape

(2838, 11)

In [383]:
print(c.Username)

skepticscience


In [384]:
full_tweets.to_pickle('{}_unique_tweets.pkl'.format(c.Username))

In [385]:
full_tweets = pd.read_pickle('{}_unique_tweets.pkl'.format(c.Username))

In [386]:
full_tweets

,id,date,time,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,username,search_term
219,1094823133977153536,2019-02-10,20:59:10,"This week, House Democrats held two hearings o...",[],0,3,14,[],skepticscience,climate change
223,1094098330718793728,2019-02-08,20:59:04,"Ever since the midterm elections, there’s been...",[],2,5,21,[],skepticscience,climate change
224,1093464425246482432,2019-02-07,03:00:09,In recent years Caribbean Community (CARICOM) ...,[],0,2,3,[],skepticscience,climate change
225,1092920830189858818,2019-02-05,15:00:06,Trump is expected to focus on five main issues...,[],0,0,1,[],skepticscience,climate change
226,1092875536119025664,2019-02-05,12:00:07,Since climate change was first introduced to U...,[],0,0,3,[],skepticscience,climate change
...,...,...,...,...,...,...,...,...,...,...,...
1,1092105203137040385,2019-02-03,08:59:05,"""I have never been an environmentalist. I don’...",[],3,9,9,[],skepticscience,environmentalist
2,991935404780785664,2018-05-03,00:00:04,The phrase “Mormon environmentalist” is not on...,[],0,1,1,[],skepticscience,environmentalist
3,944387122563354625,2017-12-22,18:00:10,Iceland's new prime minister is a feminist and...,[],1,24,47,[],skepticscience,environmentalist
5,898393806659358720,2017-08-17,20:59:09,Climate change is the biggest story on any edi...,[],3,13,13,[],skepticscience,environmentalist
